In [18]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
%%sql
drop table if exists Hotel;
CREATE TABLE Hotel (
    id integer primary key,
    floor_nbr INTEGER NOT NULL,
    room_nbr INTEGER
);

INSERT INTO Hotel VALUES(1,1, NULL);
INSERT INTO Hotel VALUES(2,1, NULL);
INSERT INTO Hotel VALUES(3,1, NULL);
INSERT INTO Hotel VALUES(4,2, NULL);
INSERT INTO Hotel VALUES(5,2, NULL);
INSERT INTO Hotel VALUES(6,3, NULL);

select *
from Hotel;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
6 rows affected.


id,floor_nbr,room_nbr
1,1,None
2,1,None
3,1,None
4,2,None
5,2,None
6,3,None


In [8]:
%%sql
-- 相関サブクエリを使用
update Hotel as h1
set room_nbr = (
    select count(*)
    from Hotel as h2
    where h1.floor_nbr = h2.floor_nbr and
    h1.id >= h2.id
);

select *
from Hotel

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.
6 rows affected.


id,floor_nbr,room_nbr
1,1,1
2,1,2
3,1,3
4,2,1
5,2,2
6,3,1


In [20]:
%%sql
-- window関数を使用(fromの中をwindow関数のサブクエリにすればOK)
update Hotel as h1
set room_nbr = (
    select idx
    from (
        select id, row_number() over (
            partition by floor_nbr
            order by id
        ) as idx
        from Hotel
    ) as h2
    where h1.id = h2.id
);

select *
from Hotel;

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.
6 rows affected.


id,floor_nbr,room_nbr
1,1,1
2,1,2
3,1,3
4,2,1
5,2,2
6,3,1
